# TP4 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

# Enunciado do Problema 2

O programa `Python` seguinte implementa o algoritmo de bubble sort para ordenação in situ de um array de inteiros `seq`.

In [1]:
seq = [-2,1,2,-1,4,-4,-3,3]
changed = True
while changed:
    changed = False
    for i in range(len(seq) - 1):
        if seq[i] > seq[i+1]:
            seq[i], seq[i+1] = seq[i+1], seq[i]
            changed = True
print(seq)

[-4, -3, -2, -1, 1, 2, 3, 4]


 1. Defina a pré-condição e a pós-condição que descrevem a especificação deste algoritmo.
 2. O ciclo `for` pode ser descrito por uma transição $\,\mathtt{seq}\gets \mathit{exp}(\mathtt{seq})\,$. Construa uma relação de transição $\;\mathsf{trans}(\mathtt{seq},\mathtt{seq}')\,$ que modele esta atribuição.
 3. Usando a técnica que lhe parecer mais conveniente verifique a correção do algoritmo.

```python
    Pré-Condição: assume n >= 0 and changed == True
    Pós-Condução: assert forall j . 0 <= j < n-1 -> seq[j] <= seq[j+1] and changed == False
```

In [2]:
from pysmt.shortcuts import *
from pysmt.typing import *

In [3]:
def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
            print(s.get_model())
        else:
            print("Proved.")

In [4]:
i = Symbol('i', INT)
array = Symbol('array', ArrayType(INT,INT))
size = Symbol('size', INT)
f = Symbol('f', FunctionType(INT,[INT, ArrayType(INT, INT), INT]))
ax1 = ForAll([array,size], Equals(f(Int(0), array, size), Select(array, Int(0))))
ax2 = ForAll([i, array, size], 
             Implies(And(i>Int(0), i<size), 
                     Equals(f(i, array, size), 
                            Ite(Select(array,i) >= f(i-Int(1), array, size), 
                                Select(array,i), f(i-Int(1), array, size)))))
axioms = And(ax1,ax2)

In [52]:
test1 = Symbol('array_test', ArrayType(INT,INT))
test2 = Symbol('inc_test', INT)
store = And(Equals(Select(test1, Int(0)), Int(1)),
            Equals(Select(test1, Int(1)), Int(0)))
test_size = Int(2)
prove(Implies(And(axioms, store), Equals(f(Int(0), test1, test_size), Int(1))))
prove(Implies(And(axioms, store), Equals(f(Int(1), test1, test_size), Int(0))))

Proved.


SolverReturnedUnknownResultError: 

In [53]:
j = Symbol('j', INT)
def for_cycle(c, p, changed, changed_p, n, n_p):
    print(c,p)
    indutivo = ForAll([j], Implies(And(j<n-Int(1), j>=Int(0)), 
                                   Equals(p, Store(p,i, Ite(Select(c, j+Int(1)) <= f(j, c, n), Select(c, j+Int(1)), f(j, c, n))))))
    
    final = Equals(p, Store(p, n - Int(1), f(n - Int(1), c, n)))
    
    #troca_true = Bool(True)
    
    #troca_false = Bool(True)
    
    #troca_true  = Iff(Iff(changed_p, Bool(True)), Exists([i], Implies(And(i>=Int(0), i<n), Not(Equals(Select(c, i), Select(p, i))))))
    
    #troca_false = Iff(Iff(changed_p, Bool(False)), ForAll([i], Implies(And(i>=Int(0), i<n), Equals(Select(c, i), Select(p, i))))) 
    
    #troca_true = Iff(Iff(changed_p, Bool(True)), Not(Equals(c,p)))
    
    troca_false = Iff(Iff(changed_p, Bool(False)), Equals(c,p))
    
    manter_n = Equals(Int(N), prime(n))
    
    return And(indutivo, final, troca_false, manter_n, axioms)

In [49]:
# Auxiliares
def prime(v):
    return Symbol("next(%s)" % v.symbol_name(), v.symbol_type())
def fresh(v):
    return FreshSymbol(typename=v.symbol_type(),template=v.symbol_name()+"_%d")

# A classe "Sigle Assignment Unfold"
class SAU(object):
    """Trivial representation of a while cycle and its unfolding."""
    def __init__(self, variables, pre , pos, control, trans, sname="z3"):
              
        self.variables = variables       # variables   
        self.pre = pre                   # pre-condition as a predicate in "variables"
        self.pos = pos                   # pos-condition as a predicate in "variables"
        self.control = control           # cycle control as a predicate in "variables"
        self.trans = trans               # cycle body as a binary transition relation 
                                         # in "variables" and "prime variables"
        
        self.prime_variables = [prime(v) for v in self.variables]
        self.frames = [And([Not(control),pos])]
                 # inializa com uma só frame: a da terminação do ciclo
        
        self.solver = Solver(name=sname)
        self.fresh = []
    def new_frame(self):        
        freshs = [fresh(v) for v in self.variables]    
        self.fresh = self.fresh + freshs
        b = self.control
        S = self.trans.substitute(dict(zip(self.prime_variables,freshs)))
        W = self.frames[-1].substitute(dict(zip(self.variables,freshs)))
        self.frames.append(And([b, ForAll(freshs, Implies(S, W))]))
        
    def unfold(self,bound=0):
        n = 0
        while True:
            if n > bound:
                print("falha: número de tentativas ultrapassa o limite %d "%bound)
                break
            #for a in self.frames:
                #print(a.serialize())
            f = Or(self.frames)
            print(Not(f).serialize())
            if self.solver.solve([self.pre,Not(f)]):
                print(self.solver.get_value(changed))
                print("seq:")
                for inc in range(N):
                    print(self.solver.get_value(Select(seq, Int(inc))))
                print("freshs:")
                for v in self.fresh:
                    if str(v.symbol_type()) == "Array{Int, Int}":
                        for inc in range(N):
                            print(v.symbol_name(), inc, ":", self.solver.get_value(Select(v, Int(inc))))
                    else:
                        print(v.symbol_name(),":", self.solver.get_value(v))
                print("end frame")
                self.new_frame()
                n += 1
            else:
                print("sucesso na tentativa %d "%n)
                break   

In [50]:
N = 8

n = Symbol('n',INT)
seq = Symbol('seq', ArrayType(INT, INT))
changed = Symbol('changed', BOOL)
variables = [changed, seq, n]

In [51]:
store1 = And(Equals(Select(seq, Int(0)), Int(-2) ),
            Equals(Select(seq, Int(1)), Int(1)  ),
            Equals(Select(seq, Int(2)), Int(2)  ),
            Equals(Select(seq, Int(3)), Int(-1) ),
            Equals(Select(seq, Int(4)), Int(4)  ),
            Equals(Select(seq, Int(5)), Int(-4) ),
            Equals(Select(seq, Int(6)), Int(-3) ),
            Equals(Select(seq, Int(7)), Int(3)  ))


pre  =  And(Equals(n, Int(N)), Iff(changed, Bool(True)), store1)  
pos  =  And(ForAll([i], Implies(And(i>=0, i<n-Int(1)), Select(seq, i) <= Select(seq, i+Int(1)))))
cond =  Iff(changed, Bool(True))               # condição de controlo do ciclo
trans = for_cycle(seq, prime(seq), changed, prime(changed), n, prime(n))

W = SAU(variables, pre, pos, cond, trans)
W.unfold(8)

seq 'next(seq)'
(! ((! (changed <-> True)) & (forall i . (((0 <= i) & (i < (n - 1))) -> (seq[i] <= seq[(i + 1)])))))
True
seq:
-2
1
2
-1
4
-4
-3
3
freshs:
end frame
(! (((! (changed <-> True)) & (forall i . (((0 <= i) & (i < (n - 1))) -> (seq[i] <= seq[(i + 1)])))) | ((changed <-> True) & (forall changed_36, seq_37, n_38 . ((((forall array, size . (f(0, array, size) = array[0])) & (forall i, array, size . (((0 < i) & (i < size)) -> (f(i, array, size) = ((f((i - 1), array, size) <= array[i]) ? array[i] : f((i - 1), array, size)))))) -> ((forall j . (((j < (n - 1)) & (0 <= j)) -> (seq_37 = seq_37[i := ((seq[(j + 1)] <= f(j, seq, n)) ? seq[(j + 1)] : f(j, seq, n))]))) & (seq_37 = seq_37[(n - 1) := f((n - 1), seq, n)]) & ((changed_36 <-> False) <-> (seq = seq_37)) & (8 = n_38))) -> ((! (changed_36 <-> True)) & (forall i . (((0 <= i) & (i < (n_38 - 1))) -> (seq_37[i] <= seq_37[(i + 1)])))))))))
True
seq:
-2
1
2
-1
4
-4
-3
3
freshs:
changed_36 : False
seq_37 0 : 0
seq_37 1 : 0
seq_37 2 : 0
s

In [10]:
store2 = And([Equals(Select(seq, Int(i)), Int(-2)) for i in range(N-1)] + [Equals(Select(seq, Int(7)), Int(-3))])
print(store2)
pre  =  And(Equals(n, Int(N)), Iff(changed, Bool(True)), store2)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

((seq[0] = -2) & (seq[1] = -2) & (seq[2] = -2) & (seq[3] = -2) & (seq[4] = -2) & (seq[5] = -2) & (seq[6] = -2) & (seq[7] = -3))
(! ((! (changed <-> True)) & (forall i . (((0 <= i) & (i < (n - 1))) -> (seq[i] <= seq[(i + 1)])))))
True
seq:
-2
-2
-2
-2
-2
-2
-2
-3
freshs:
end frame
(! (((! (changed <-> True)) & (forall i . (((0 <= i) & (i < (n - 1))) -> (seq[i] <= seq[(i + 1)])))) | ((changed <-> True) & (forall changed_3, seq_4, n_5 . (((forall j . (((j < (n - 1)) & (0 <= j)) -> (seq_4[i] = ((seq[(j + 1)] <= f(j, seq, n)) ? seq[(j + 1)] : f(j, seq, n))))) & (seq_4[(n - 1)] = f((n - 1), seq, n)) & ((changed_3 <-> False) <-> (seq = seq_4)) & (8 = n_5) & ((forall array, size . (f(0, array, size) = array[0])) & (forall i, array, size . (((0 < i) & (i < size)) -> (f(i, array, size) = ((f((i - 1), array, size) <= array[i]) ? array[i] : f((i - 1), array, size))))))) -> ((! (changed_3 <-> True)) & (forall i . (((0 <= i) & (i < (n_5 - 1))) -> (seq_4[i] <= seq_4[(i + 1)])))))))))
sucesso na tenta

In [11]:
store3 = And([Equals(Select(seq, Int(i)), Int(-2)) for i in range(N)])
print(store3)
pre  =  And(Equals(n, Int(N)), Iff(changed, Bool(True)), axioms, store3)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

((seq[0] = -2) & (seq[1] = -2) & (seq[2] = -2) & (seq[3] = -2) & (seq[4] = -2) & (seq[5] = -2) & (seq[6] = -2) & (seq[7] = -2))
(! ((! (changed <-> True)) & (forall i . (((0 <= i) & (i < (n - 1))) -> (seq[i] <= seq[(i + 1)])))))


SolverReturnedUnknownResultError: 

In [38]:
#test = []
#for inc in range(N-1):
#    test.append(Equals(f(Int(inc)), Int(-2)))
test = Bool(True)
N = 8
pre  =  And(Equals(n, Int(N)), Iff(changed, Bool(True)), store3)
t = for_cycle(seq, prime(seq), changed, prime(changed), n, prime(n))
with Solver(name="z3") as solver:
    if solver.solve([t, pre]):
        print(store3.simplify().serialize())
        print()
        print(t.simplify().serialize())
        print()
        print(pre.simplify().serialize())
        print(And(test).simplify().serialize())
        print("Oh")
        #print(solver.get_value(seq))
        for inc in range(N):
            print(solver.get_value(Select(seq, Int(inc))))
        print("diff")
        for inc in range(N):
            print(solver.get_value(Select(prime(seq), Int(inc))))
        print(solver.get_value(changed))
        print(solver.get_value(prime(changed)))
    else:
        print("Yo")

seq 'next(seq)'


SolverReturnedUnknownResultError: 

In [55]:
N = 5
store4 = And(Equals(Select(seq, Int(0)), Int(1)),
             Equals(Select(seq, Int(1)), Int(2)),
             Equals(Select(seq, Int(2)), Int(0)),
             Equals(Select(seq, Int(3)), Int(1)),
             Equals(Select(seq, Int(4)), Int(3)))
t = for_cycle(seq, prime(seq), changed, prime(changed), n, prime(n))
pre  =  And(Equals(n,Int(N)), Iff(changed, Bool(True)), store4)
with Solver(name="z3") as solver:
    if solver.solve([t, pre]):
        print(t.simplify().serialize())
        print()
        print(pre.simplify().serialize())
        print("Oh")
        #print(solver.get_value(seq))
        for inc in range(N):
            print(solver.get_value(Select(seq, Int(inc))))
        print("diff")
        for inc in range(N):
            print(solver.get_value(Select(prime(seq), Int(inc))))
        print(solver.get_value(changed))
        print(solver.get_value(prime(changed)))
    else:
        print("Yo")

seq 'next(seq)'
(((forall i, array, size . (((0 < i) & (i < size)) -> (f(i, array, size) = ((f((i - 1), array, size) <= array[i]) ? array[i] : f((i - 1), array, size))))) & (forall array, size . (f(0, array, size) = array[0]))) -> ((forall j . (((j < (n - 1)) & (0 <= j)) -> ('next(seq)' = 'next(seq)'[i := ((seq[(j + 1)] <= f(j, seq, n)) ? seq[(j + 1)] : f(j, seq, n))]))) & ('next(seq)' = 'next(seq)'[(n - 1) := f((n - 1), seq, n)]) & ((! 'next(changed)') <-> (seq = 'next(seq)')) & (5 = 'next(n)')))

((seq[0] = 1) & (seq[1] = 2) & (seq[2] = 0) & (seq[3] = 1) & changed & (seq[4] = 3) & (n = 5))
Oh
1
2
0
1
3
diff
14
14
14
14
14
True
False


In [ ]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)